In [19]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [20]:
!git clone https://github.com/gerarldoindra/palm-oil-dataset

fatal: destination path 'palm-oil-dataset' already exists and is not an empty directory.


In [21]:
pip install -r /content/palm-oil-dataset/req/requirements.txt

In [22]:
import torch
from yolov5 import utils
import torch
from IPython import display
from IPython.display import clear_output
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import io
import os
import cv2
import json
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

%matplotlib inline

In [23]:
PROJECT_NAME = "yolov5_train"
BASE_MODEL = "yolov5m6.pt"
TRAIN_BATCH = 32
TRAIN_EPOCHS = 200
VAL_BATCH = 64
NOTES_PATH = "/content/palm-oil-dataset/notes.json"

In [24]:
descr_darknet = json.load(open(NOTES_PATH))

train_path = "/content/palm-oil-dataset/train/images"
test_path = "/content/palm-oil-dataset/test/images"
valid_path = "/content/palm-oil-dataset/valid/images"

nc = len(descr_darknet["categories"])
names = [category["name"] for category in descr_darknet["categories"]]

print(
    f"train: {train_path}\n"
    f"test: {test_path}\n"
    f"val: {valid_path}\n\n"
    f"nc: {nc}\n"
    f"names: {names}",
)

train: /content/palm-oil-dataset/train/images
test: /content/palm-oil-dataset/test/images
val: /content/palm-oil-dataset/valid/images

nc: 7
names: ['Ripe', 'Unripe', 'overripe', 'ripe', 'ripe', 'underripe', 'unripe']


In [25]:
with open("data.yaml", "w") as file:
    yaml.dump({
        "train": train_path,
        "test": test_path,
        "val": valid_path,
        "nc": nc,
        "names": [f'{name}' for name in names]
    }, stream=file, default_flow_style=None)

In [26]:
wildcard = f"{PROJECT_NAME}/feature_extraction*"
! rm -r $wildcard

In [ ]:
! python yolov5/train.py --batch $TRAIN_BATCH --epochs $TRAIN_EPOCHS --data "data.yaml" --weights $BASE_MODEL --project $PROJECT_NAME --name 'feature_extraction' --cache --freeze 12

train: weights=yolov5m6.pt, cfg=, data=data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5_train, name=feature_extraction, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[12], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-227-ge4df1ec Python-3.10.12 torch-2.1.0+cu118 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, 

In [ ]:
# Delete old results
wildcard = f"{PROJECT_NAME}/validation_on_test_data*"
! rm -r $wildcard

In [ ]:
WEIGHTS_BEST = f"{PROJECT_NAME}/feature_extraction/weights/best.pt"
! python yolov5/val.py --weights $WEIGHTS_BEST --batch $VAL_BATCH --data 'data.yaml' --task test --project $PROJECT_NAME --name 'validation_on_test_data' --augment

In [ ]:
# Delete old results
wildcard = f"{PROJECT_NAME}/detect_test*"
! rm -r $wildcard

In [ ]:
! python yolov5/detect.py --weights $WEIGHTS_BEST --conf 0.6 --source 'test/images' --project $PROJECT_NAME --name 'detect_test' --augment --line=3

In [ ]:
def read_images(dirpath):
  images = []
  for img_filename in os.listdir(dirpath):
    images.append(mpimg.imread(f"{dirpath}/{img_filename}"))
  return images

In [ ]:
def label_test_images(test_images_path, test_labels_path, classes):
  test_images = os.listdir(test_images_path)
  labeled_images = []

  for idx, test_image_filename in enumerate(test_images):
    image = mpimg.imread(f"{test_images_path}/{test_image_filename}")

    x_shape, y_shape = image.shape[1], image.shape[0]

    test_label_filename = f"{test_image_filename[:-4]}.txt"

    with open(f"{test_labels_path}/{test_label_filename}", "r") as f:
      lines = f.readlines()

      for line in lines:
        # Parse line
        box = line.split()
        class_idx = box[0]

        class_name = names[int(class_idx)]
        x_center, y_center, box_w, box_h = int(float(box[1])*x_shape), int(float(box[2])*y_shape), int(float(box[3])*x_shape), int(float(box[3])*y_shape)
        x1, y1, x2, y2 = x_center-int(box_w/2), y_center-int(box_h/2), x_center+int(box_w/2), y_center+int(box_h/2)

        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
        cv2.putText(image, class_name, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 3)

    labeled_images.append(image)

  return labeled_images

In [ ]:
detect_path = f"{PROJECT_NAME}/detect_test"
test_images_path = f"/content/palm-oil-dataset/test/images"
test_labels_path = f"/content/palm-oil-dataset/test/labels"

detected_images = read_images(detect_path)
test_labeled_images = label_test_images(test_images_path, test_labels_path, classes=names)

stacked_images = [np.hstack([detected_images[idx], test_labeled_images[idx]]) for idx in range(len(detected_images))]

In [ ]:
for image in stacked_images:
  fig = plt.figure(figsize=(40, 15))
  ax1 = fig.add_subplot(2,2,1)
  ax1.imshow(image)